# import

In [23]:
import numpy as np
import pandas as pd
import os
from IPython.display import display, HTML

def get_dlc_data(Tracking_data_path,interp,val):
    # Load in '.h5' file:
    h5_read=pd.read_hdf(Tracking_data_path)
    # Access the head center      
    scorer =  h5_read.columns.tolist()[0][0]

    colum_headings = h5_read[scorer].columns
    bodyparts = np.unique([item[0] for item in colum_headings])

    output = {}
    for name in bodyparts:
        print(name)
        dat_ =  h5_read[scorer][name]
        if interp:
            dat_interped=clean_and_interpolate(dat_,val)
        output[name] =[dat_interped]
    return output


def clean_and_interpolate(data,threshold):

    bad_confidence_inds = np.where(data.likelihood.values<threshold)[0]
    newx = data.x.values
    newx[bad_confidence_inds] = 0
    newy = data.y.values
    newy[bad_confidence_inds] = 0

    start_value_cleanup(newx)
    interped_x = interp_0_coords(newx)

    start_value_cleanup(newy)
    interped_y = interp_0_coords(newy)
    
    data['interped_x'] = interped_x
    data['interped_y'] = interped_y
    
    return data

def start_value_cleanup(coords):
    # This is for when the starting value of the coords == 0; interpolation will not work on these coords until the first 0 
    #is changed. The 0 value is changed to the first non-zero value in the coords lists
    for index, value in enumerate(coords):
        working = 0
        if value > 0:
            start_value = value
            start_index = index
            working = 1
            break
    if working == 1:
        for x in range(start_index):
            coords[x] = start_value
            
def interp_0_coords(coords_list):
    #coords_list is one if the outputs of the get_x_y_data = a list of co-ordinate points
    for index, value in enumerate(coords_list):
        if value == 0:
            if coords_list[index-1] > 0:
                value_before = coords_list[index-1]
                interp_start_index = index-1
                

        if index < len(coords_list)-1:
            if value ==0:
                if coords_list[index+1] > 0:
                    interp_end_index = index+1
                    value_after = coords_list[index+1]
                   

                    #now code to interpolate over the values
                    try:
                        interp_diff_index = interp_end_index - interp_start_index
                    except UnboundLocalError:
#                         print('the first value in list is 0, use the function start_value_cleanup to fix')
                        break
                 

                    new_values = np.linspace(value_before, value_after, interp_diff_index)
                    #print(new_values)

                    interp_index = interp_start_index+1
                    for x in range(interp_diff_index):
                        
                        coords_list[interp_index] = new_values[x]
                        interp_index +=1
        if index == len(coords_list)-1:
            if value ==0:
                for x in range(30):
                    coords_list[index-x] = coords_list[index-30]
                    #print('')
    print('function exiting')
    return(coords_list)

# main

In [ ]:
# load in behavioural data from first and second day and extract relevant information

# load in replay data from first day and extract relevant information

In [1]:
behaviour_path = r'Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\'

In [ ]:


def load_in_behavioural_data(current_path):

    day2_tracking = False
    port_tracking2,movement_tracking2,port_tracking1,movement_tracking1 = None, None, None, None
    
    for day_folder in os.listdir(current_path):
        day_path = os.path.join(current_path, day_folder)
        if 'day2' in day_path:
            print(f'Processing {day_folder}')
            for file in os.listdir(day_path):
                if file.endswith('.csv'):
                    df_2 = pd.read_csv(os.path.join(day_path,file))
                if file.endswith('.h5'):
                    print('day 2 tracking found')
                    day2_tracking = True
                    # tracking exists
                    if 'port' or 'PORT' in file:
                        port_tracking2 = get_dlc_data(os.path.join(day_path, file),interp = True,val = 0.9995)
                    else:
                        movement_tracking2 = get_dlc_data(os.path.join(day_path, file),interp = True,val = 0.9995)                        
    for day_folder in os.listdir(current_path):
        day_path = os.path.join(current_path, day_folder)
        if 'day1' in day_path:
            print(f'Processing {day_folder}')
            for file in os.listdir(day_path):
                if file.endswith('.csv'):
                    df_1 = pd.read_csv(os.path.join(day_path,file))
                if day2_tracking:
                    # if day 2 tracking exists, then we also load in the day 1 tracking
                    if file.endswith('.h5'):
                        'day 1 tracking found'
                        # tracking exists
                        if 'port' or 'PORT' in file:
                            port_tracking1 = get_dlc_data(os.path.join(day_path, file),interp = True,val = 0.9995)
                        else:
                            movement_tracking1 = get_dlc_data(os.path.join(day_path, file),interp = True,val = 0.9995)

    return df_1, df_2, movement_tracking1, movement_tracking2, port_tracking1, port_tracking2




for m_i_r in os.listdir(behaviour_path):
    current_path = os.path.join(behaviour_path, m_i_r)
    display(HTML(f"<b>Processing behavioural data for: {m_i_r}</b>"))
    df_1, df_2, movement_tracking1, movement_tracking2, port_tracking1, port_tracking2 = load_in_behavioural_data(current_path)
    

Processing day2_17-11-2024
Processing day1_16-11-2024


Processing day2_19-11-2024
day 2 tracking found


In [34]:
file

'PreProcessed_RawPokeData.csv'

In [26]:
day_path

'Z:\\projects\\sequence_squad\\revision_data\\emmett_revisions\\sleep_wake_link_data\\replay_to_behaviour\\\\AP5R_1_1\\day2_17-11-2024'

In [19]:
os.listdir(day_path)

['PreProcessed_RawPokeData.csv']

In [18]:
day_path

'Z:\\projects\\sequence_squad\\revision_data\\emmett_revisions\\sleep_wake_link_data\\replay_to_behaviour\\\\SEQ008_1_3\\day2_14-11-2024'

In [17]:
os.listdir(day_path)

['PreProcessed_RawPokeData.csv']

In [16]:
file

NameError: name 'file' is not defined

In [8]:
current_path

'Z:\\projects\\sequence_squad\\revision_data\\emmett_revisions\\sleep_wake_link_data\\replay_to_behaviour\\\\SEQ008_1_3'

['PreProcessed_RawPokeData.csv']